<a href="https://colab.research.google.com/github/Cycadophyta/qpcr-analysis/blob/main/Cankerlink_qPCR_analysis_WP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cankerlink qPCR Analysis WP1

In [ ]:
# Packages
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as pltyu
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
from plotly.subplots import make_subplots
from scipy import stats

## Load Data

Each plate is stored in a dictionary with metadata.


* Name: plate name
* Data: Cq data
* Samples: datafraframe of sample number and corresponding ID
* Dilution: dilution factor for plate
* Wells to remove: a list of wells to exclude from analysis



In [ ]:
plate_1 = {
    'name': 'Plate 1 (ITS)',
    'data': pd.read_csv('WP1_Plate_1_ITS_Cq.csv'),
    'samples': pd.read_csv('Samples - Plate 1&2.csv'),
    'dilutions': {'a': 5, 'b': 10},
    'wells_to_remove': []
}

plate_2 = {
    'name': 'Plate 2 (16S)',
    'data': pd.read_csv('WP1_Plate_2_16S_Cq.csv'),
    'samples': pd.read_csv('Samples - Plate 1&2.csv'),
    'dilutions': {'a': 5, 'b': 10},
    'wells_to_remove': []
}

plate_3 = {
    'name': 'Plate 3 (ITS)',
    'data': pd.read_csv('WP1_Plate_3_ITS_Cq.csv'),
    'samples': pd.read_csv('Samples - Plate 3&4.csv'),
    'dilutions': {'a': 10, 'b': 50},
    'wells_to_remove': []
}

plate_4 = {
    'name': 'Plate 4 (16S)',
    'data': pd.read_csv('WP1_Plate_4_16S_Cq.csv'),
    'samples': pd.read_csv('Samples - Plate 3&4.csv'),
    'dilutions': {'a': 10, 'b': 50},
    'wells_to_remove': ['F17', 'C09', 'K21']
}

plate_5 = {
    'name': 'Plate 5 (ITS)',
    'data': pd.read_csv('WP1_Plate_5_ITS_Cq.csv'),
    'samples': pd.read_csv('Samples - Plate 5&6.csv'),
    'dilutions': {'a': 10, 'b': 101},
    'wells_to_remove': ['G02']
}

plate_6 = {
    'name': 'Plate 6 (16S)',
    'data': pd.read_csv('WP1_Plate_6_16S_Cq.csv'),
    'samples': pd.read_csv('Samples - Plate 5&6.csv'),
    'dilutions': {'a': 10, 'b': 101},
    'wells_to_remove': ['C04', 'I02', 'K17']
}

plate_7 = {
    'name': 'Plate 7 (ITS)',
    'data': pd.read_csv('WP1_Plate_7_ITS_Cq.csv'),
    'samples': pd.read_csv('Samples - Plate 7&8.csv'),
    'dilutions': {'a': 10, 'b': 51},
    'wells_to_remove': []
}

plate_8 = {
    'name': 'Plate 8 (16S)',
    'data': pd.read_csv('WP1_Plate_8_16S_Cq.csv'),
    'samples': pd.read_csv('Samples - Plate 7&8.csv'),
    'dilutions': {'a': 10, 'b': 51},
    'wells_to_remove': []
}

standard_concentrations = {
    'P1': math.log10(5e-1),
    'P2': math.log10(5e-2),
    'P3': math.log10(5e-3),
    'P4': math.log10(5e-4),
    'P5': math.log10(5e-5),
}

plates = [plate_1, plate_2, plate_3, plate_4, plate_5, plate_6, plate_7, plate_8]

## Data Processing

Various functions to clean and transform the data and to perform calculations.

'data' is cleaned and undecessary columns dropped. Standards and NTCs are removed and made into their own dataframes, 'standards' and 'ntcs'. 'detection_limit' is calculated from the mean of the NTC Cq values and Cq values less than the detection limit are replaced by NaN in 'data'. 'tidy_data' is a summary which includes the mean and standard deviation of replicates. 'tidy_data' also includes the efficiency between dilutions. 

Efficiency calculated as: 

$Efficiency = -1+10^\frac{-1}{slope}$

Starting quantity (SQ) calculated as: 

$SQ = dilution factor * 10 ^ \frac{Cq - intercept} {slope}$

In [ ]:
def clean_data(plate):
    '''Reformats the dataframe into a more usable format.'''  
    data = plate['data']    
    samples = plate['samples']              
    data.rename(columns = {'Biological Set Name': 'Dilution'}, inplace=True)
    data.replace({'Dilution': plate['dilutions']}, inplace=True)
    data = data.astype({'Sample': str})
    samples.rename(columns={'Tube': 'Sample'}, inplace=True)
    samples = samples.astype({'Sample': str})
    data = pd.merge(data, samples, on='Sample', how='left') 
    plate['data'] = data[['Well', 'Content', 'Sample','ID', 'Dilution', 'Cq']]  
    plate['samples'] = samples       


def timepoints(plate):
    data = plate['data']
    data['timepoint'] = np.where(data.ID.str[:2] == 'WP', data.ID.str[5], '1')
    plate['data'] = data   
    

def extract_controls(plate):
    '''Extracts standards and NTCs to their own dataframes.'''
    standard_samples = ['P1', 'P2', 'P3', 'P4', 'P5']
    data = plate['data']

    plate['standards'] = data[
        data.Sample.isin(standard_samples)
    ][['Well', 'Sample', 'Cq']].reset_index()
    
    plate['ntcs'] = data[
        data.Content == 'NTC'
    ][['Well', 'Content', 'Cq']].reset_index()
    
    plate['data'] = data.loc[
        ~((data.Sample.isin(standard_samples)) | (data.Content == 'NTC')), :
    ].reset_index()


def calculate_detection_limit(plate):
    '''Calculates the detection limit of each plate from the mean of 
    the NTC Cq values and replaces Cq values above this limit with NaN.
    '''
    plate['detection_limit'] = round(plate['ntcs'].Cq.mean(), 2)

    plate['data'].Cq.where((
        plate['data'].Cq < plate['detection_limit']
    ), np.NaN, inplace=True)


def tidy_data(plate):
    '''Calculates mean and std of technical replicates.'''
    columns = ['Sample', 'ID', 'Content', 'Dilution', 'Cq', 'quantity']
    plate['tidy_data'] = plate['data'].groupby(
        ['Sample', 'ID', 'Dilution', 'Content']
    ).agg(
        Cq_mean = ('Cq', 'mean'),
        Cq_SD = ('Cq', 'std'),
        SQ_mean = ('quantity', 'mean'),
        SQ_SD = ('quantity', 'std')
    ).reset_index().drop(columns=['Content'])


def standard_line(plate):
    '''Calculates the standard regression line.'''
    standards = plate['standards']
    standards['concentration'] = standards['Sample'].map(standard_concentrations)
    x = standards.concentration
    y = standards.Cq
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    plate['standard_line'] = {
        'slope': slope, 'intercept': intercept, 'r_value': r_value
    }


def calculate_quantity(plate):
    '''Calculates starting quantity based on the formula:
    dilution * 10 ^ ((Cq - intercept) / slope)
    '''
    data = plate['data']
    intercept = plate['standard_line']['intercept']
    slope = plate['standard_line']['slope']
    data['quantity'] = data.Dilution * 10 ** ((data.Cq - intercept) / slope)


def remove_wells(plate):
    '''Removes wells marked as outliers.'''
    df = plate['data']
    df.Cq = df.Cq.where(~df.Well.isin(plate['wells_to_remove']))


def calculate_efficiency(plate):
    '''Calculates efficiency between dilutions based on the formula:
     -1 + 10 ^ -(1 / slope)
     and appends 
    values to tidy_data.
    '''
    df = plate['tidy_data'].pivot(
        index='ID', columns='Dilution', values='Cq_mean'
    ).reset_index()

    def efficiency(a, b, d1, d2):
        '''Formula for efficiency.'''
        slope = (a-b)/(d1-d2)
        return -1 + 10 ** -(1/slope)
    
    a, b = plate['dilutions']['a'], plate['dilutions']['b']
    df['Efficiency'] = efficiency(
        df.iloc[:, 1], df.iloc[:, 2], 
        math.log10(1/a), math.log10(1/b)
    )
    df = df.set_index('ID')['Efficiency']
    plate['tidy_data'] = plate['tidy_data'].join(df, on='ID')


def process_data(plate):
    '''Main data processing function that executes all 
    data processing functions.
    '''
    try:
        clean_data(plate)
        timepoints(plate)
        #remove_wells(plate)
        extract_controls(plate)
        calculate_detection_limit(plate)
        standard_line(plate)
        calculate_quantity(plate)
        tidy_data(plate)
        calculate_efficiency(plate)
        print(f"{plate['name']} data processed sucessfully.")
    except:
        if 'standards' in plate:
            print(f"{plate['name']} data has already been processed.")
        else: print('Unknown error')


for plate in plates: process_data(plate)

Plate 1 (ITS) data processed sucessfully.
Plate 2 (16S) data processed sucessfully.
Plate 3 (ITS) data processed sucessfully.
Plate 4 (16S) data processed sucessfully.
Plate 5 (ITS) data processed sucessfully.
Plate 6 (16S) data processed sucessfully.
Plate 7 (ITS) data processed sucessfully.
Plate 8 (16S) data processed sucessfully.


In [ ]:
joined = pd.concat([plate['data'] for plate in plates])

#display(joined[1000: 2000])

joined.to_csv('WP1_full_data.csv', index=False)

## Figures

### NTCs

In [ ]:
def view_ntcs(plate):
    '''Generates plotly express bar graphs of NTC Cq values.'''
    return px.bar(
        plate['ntcs'], 
        x = 'Well', 
        y = "Cq", 
        title = f"{plate['name']} NTCs"
    )

#view_ntcs(plate_1).show()
#view_ntcs(plate_2).show()
#view_ntcs(plate_3).show()
#view_ntcs(plate_4).show()
#view_ntcs(plate_5).show()
#view_ntcs(plate_6).show()
view_ntcs(plate_7).show()
view_ntcs(plate_8).show()

### Standards

In [ ]:
def view_standards(plate):
    '''Generate standard curves.'''
    table = plate['standards'].groupby(['Sample']).agg(
        Mean_Cq = ('Cq', 'mean'),
        SD = ('Cq', 'std')
    ).rename_axis(None)
    fig = px.scatter(
        plate['standards'], 
        x='Log(SQ)', y="Cq", 
        trendline='ols', 
        title=f"{plate['name']} standard curve"
    )
    fig.add_trace(go.Scatter())
    display(table)
    return fig

#view_standards(plate_1).show()
#view_standards(plate_2).show()
#view_standards(plate_3).show()
#view_standards(plate_4).show()
#view_standards(plate_5).show()
view_standards(plate_6).show()
#view_standards(plate_7).show()
#view_standards(plate_8).show()

### Cq

In [ ]:
def view_cq(plate):
    return px.scatter(
        plate['data'], 
        x='ID', y='Cq', 
        color='Dilution', 
        title=f"{plate['name']} Cq values"
    )

#view_cq(plate_1).show()
#view_cq(plate_2).show()
#view_cq(plate_3).show()
#view_cq(plate_4).show()
view_cq(plate_5).show()
view_cq(plate_6).show()
#view_cq(plate_7).show()
#view_cq(plate_8).show()

### Standard deviation of replicates

In [ ]:
def rep_sd(plate, limit=0.5):
    df = tidy_data(plate)
    return px.box(df, x='Dilution', y='Cq_SD', points='all', 
                 hover_data=['ID', 'Cq_mean'], 
                 title=f"{plate['name']} Cq standard deviation")
    
#rep_sd(plate_1).show()
#rep_sd(plate_2).show()
#rep_sd(plate_3).show()
#rep_sd(plate_4).show()
#rep_sd(plate_5).show()
#rep_sd(plate_6).show()
#rep_sd(plate_7).show()
#rep_sd(plate_8).show()


def quantity_sd(plate, limit=0.5):
    df = tidy_data(plate)
    return px.box(df, x='Dilution', y='SQ_SD', points='all', 
                 hover_data=['ID', 'SQ_mean'], 
                 title=f"{plate['name']} SQ standard deviation")
    
quantity_sd(plate_1).show()
quantity_sd(plate_2).show()
quantity_sd(plate_3).show()
quantity_sd(plate_4).show()
quantity_sd(plate_5).show()
quantity_sd(plate_6).show()
quantity_sd(plate_7).show()
quantity_sd(plate_8).show()

### Efficiency

In [ ]:
def view_efficiency(plate):
    '''Generates boxplots of efficiencies.'''
    df = calculate_efficiency(plate)
    fig = px.box(df, y="Efficiency", points='all', hover_data=['ID'],
                 title=f"{plate['name']} Efficiency")
    return fig
    

#view_efficiency(plate_1).show()
#view_efficiency(plate_2).show()
#view_efficiency(plate_3).show()
#view_efficiency(plate_4).show()
#view_efficiency(plate_5).show()
#view_efficiency(plate_6).show()
#view_efficiency(plate_7).show()
#view_efficiency(plate_8).show()

### Starting Quantity (SQ)

In [ ]:
def view_quantity(plate):
    return px.scatter(
        plate['data'], 
        x='ID', y='quantity', 
        color='Dilution',
        title=f"{plate['name']} Starting Quantity"
    )

view_quantity(plate_7).show()
view_quantity(plate_8).show()

## Repeats

In [ ]:
def repeats(plate, sd_limit, efficiency_range):
    data = plate['tidy_data']
    return data[
        (data.Cq_SD > sd_limit) | 
        ~(data.Efficiency.between(efficiency_range[0], efficiency_range[1]))
    ]

for plate in plates: 
    reps = repeats(plate, 0.5, [0.5, 1.5])
    number = len(reps.ID.drop_duplicates())
    print(plate['name'])
    print('Repeats: ', number)
    if number > 0: display(reps)
    print()


In [ ]:
left = plate_1['data']
right = plate_7['data']
left = left[left.ID.isin(right.ID)]
right = right[right.ID.isin(left.ID)]

combined = pd.concat([left, right], keys=['old', 'new']).reset_index()


print(combined)

def view_repeats(df):
    return px.scatter(
        df, 
        x='ID', y='quantity', 
        color='level_0', 
        symbol='Dilution'
        #title=f"{plate['name']} Cq values"
    )

view_repeats(combined).show()

    level_0  level_1 Well   Content  ... Dilution         Cq          ID  quantity
0       old        4  A05  Unkn-009  ...      5.0  17.750000    WP1-t3-6  1.929513
1       old       21  A22  Unkn-026  ...      5.0  17.270000   WP1-t3-27  2.539666
2       old       28  B05  Unkn-009  ...      5.0  17.950000    WP1-t3-6  1.720788
3       old       45  B22  Unkn-026  ...      5.0  17.670000   WP1-t3-27  2.019928
4       old       52  C05  Unkn-033  ...     10.0  19.700000    WP1-t3-6  1.263876
..      ...      ...  ...       ...  ...      ...        ...         ...       ...
99      new      312  P01  Unkn-150  ...     51.0  21.943260  WP1-t3-140  1.927908
100     new      313  P02  Unkn-152  ...     51.0  21.095338  WP1-t3-142  3.063505
101     new      314  P03  Unkn-154  ...     51.0  24.690554  WP1-t3-150  0.429944
102     new      315  P04  Unkn-156  ...     51.0  23.931214  WP1-t3-151  0.650927
103     new      316  P05  Unkn-158  ...     51.0  20.803504  WP1-t3-152  3.592885

[10

In [ ]:
def remove_repeats(plate_a, plate_b):
    plate_a['data'] = plate_a['data'][~plate_a['data'].ID.isin(plate_b['data'].ID)]

remove_repeats(plate_1, plate_7)
remove_repeats(plate_2, plate_8)


its_combined = concat([plate_1['data'], plate_3['data'], plate_5['data'], plate_7['data'], ])


## Outliers

In [ ]:
def efficiency_outliers(plate, upper, lower):
    '''Returns a list of sample IDs with efficiency out of bounds.'''
    df = calculate_efficiency(plate)
    return df[~df.Efficiency.between(lower, upper)]#.dropna()#['ID', 'Efficie']

print(efficiency_outliers(plate_1, 1.5, 0.5))

In [ ]:
def quantity_outliers(plate):
    df = plate['data']
    upper = 0.95
    lower = 0.05

    outliers = df[df.groupby('Sample').quantity.transform(
        lambda x : (x<x.quantile(upper))&(x>(x.quantile(lower)))
    ).eq(1)]

    df.quantity = df.quantity.where(df.quantity.isin(outliers.quantity))



#def remove_outliers(plate):

#print(plate_1['data'])
#quantity_outliers(plate_1)
#print(plate_1['data'])

## New Plots

In [ ]:
px.bar(
        plate['ntcs'], 
        x = 'Well', 
        y = "Cq", 
        title = f"{plate['name']} NTCs"
    )

px.scatter(
        plate['standards'], 
        x='Log(SQ)', y="Cq", 
        trendline='ols', 
        title=f"{plate['name']} standard curve"
    )

In [ ]:
print(plate_1['ntcs'].Cq)

In [ ]:
fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3])

fig.add_trace(go.Scatter(x=[1, 2, 3], y=[4, 5, 6]),
              row=1, col=1)

fig.add_trace(go.Scatter(x=[20, 30, 40], y=[50, 60, 70]),
              row=1, col=2)

fig.show()

## Plots

In [ ]:
def ntcs(plate):
    ntcs = plate['ntcs']
    return go.Bar(x=ntcs.Well, y=ntcs.Cq)

def standards(plate):
    standards = plate['standards']
    return go.Scatter(
        x=standards['Log(SQ)'], 
        y=standards["Cq"],
        mode='markers'
    )

## Interactive

In [ ]:
plates = [plate_1, 
          plate_2, 
          plate_3, 
          plate_4, 
          plate_5, 
          plate_6, 
          plate_7, 
          plate_8]

fig = make_subplots(
    rows=1, cols=2, column_widths=[0.5, 0.5],
    subplot_titles=('NTCs', 'Standards')
)
fig.update_xaxes(title_text='Well', row=1, col=1)
fig.update_yaxes(title_text='Cq', row=1, col=1)
fig.update_xaxes(title_text='Log(SQ)', row=1, col=2)
fig.update_yaxes(title_text='Cq', row=1, col=2)

for plate in plates:
    trace = ntcs(plate)
    fig.append_trace(trace, 1, 1)

for plate in plates:
    trace = standards(plate)
    fig.append_trace(trace, 1, 2)


labels = [plate['name'] for plate in plates]

buttons = []
for i, label in enumerate(labels):
    visibility = [i==j for j in range(len(labels))]
    button = dict(
        label =  label,
        method = 'update',
        args = [
            {'visible': visibility},
            #{'title': label}
        ])
    buttons.append(button)

updatemenus = list([dict(active=-1, x=0.13, y=1.1, buttons=buttons)])

fig.layout.title = 'Multiplot'
fig.layout.showlegend = False
fig.layout.updatemenus = updatemenus

fig.show()